In [3]:
import pandas as pd
import numpy as np
import plotnine

Queries:
- Given that the reflectance 
- Is the means of deciding wv sensitivity range optimal?

TriOS units are in $mW/(m^2 nm Sr)$, so convert to W later.

In [11]:
df = pd.read_table('spec_data_uncurated.txt',delimiter='\t')
df.specradW = df.specrad * 10**3
df %>% select(-X1,-specrad) -> df
df$specradW[df$specradW<0] <- 0 # values where the radiance is given as below zero are made zero
df %>% filter(between(wv,350,750)) -> df # drop irrelevant and uncertain values
df$white <- (df$dark / 100) - 0.5
df$white <- (-df$white) + 0.5

SyntaxError: invalid syntax (<ipython-input-11-b60fe0e130f7>, line 3)

In [16]:
df = pd.read_table('spec_data_uncurated.txt', sep='\t')

FileNotFoundError: [Errno 2] File spec_data_uncurated.txt does not exist: 'spec_data_uncurated.txt'

In [15]:
ls

 Volume in drive C is OS
 Volume Serial Number is 1CA0-5675

 Directory of C:\Users\john\GitHub\P_lividus_adult_vision\src

2020-05-16  21:52    <DIR>          .
2020-05-16  21:52    <DIR>          ..
2020-05-16  19:25    <DIR>          .ipynb_checkpoints
2020-05-13  17:58         4ÿ934ÿ114 DoG_40_150dpi.tiff
2020-05-13  17:46         4ÿ934ÿ114 DoG40_150dpi.tiff
2020-03-10  10:22             1ÿ266 downwelling_radiance.Rmd
2020-03-10  10:22             2ÿ577 extract_spec_data.py
2020-05-07  12:28             1ÿ817 invertfit.rds
2020-05-16  18:48               631 material_reflectance.csv
2020-05-16  12:40         7ÿ433ÿ922 Mex_100_150dpi.tiff
2020-05-14  14:13         8ÿ207ÿ862 Mex_120_150dpi.tiff
2020-05-13  17:59         4ÿ561ÿ524 Mex_40_150dpi.tiff
2020-04-27  11:07             4ÿ217 P_lividus_adult_vision_DOE.Rmd
2020-03-10  10:22             9ÿ854 pre-analysis.Rmd
2020-05-14  13:55             2ÿ925 reflectance in differing windows.Rmd
2020-05-07  14:18             1ÿ801 reflectanc

Get the bin width of wavelengths in nm.

In [6]:
binwidth = np.round(sum(diff(sort(unique(df.wv))))/len(diff(sort(unique(df.wv)))),2)
print 'The bin width is'+str(binwidth)+'nm'

SyntaxError: invalid syntax (<ipython-input-6-44758fd655f6>, line 1)

The test pieces for ink that I have used are marked 0 to 100 getting darker
(more ink) in steps of 5, indicated by the factor dark. I have added a converse
factor, white which runs from 0 to 1 in steps of 0.05, getting brighter. This is
to have a factor with a 0-1 scale which scales positively with quanta, which is
intuitive and simpler to model. 

I neglected to record the values for the darkest test piece with the thicker
material. However, the differences in reflectance between the smallest values are
infinitesimal (see below), so this loses a datapoint but will not have a major
(or even recognisable) impact on the printed patterns. 

## Photon radiance

The energy of a photon (joules) - in terms of wavelength (\lambda) - is defined as: $$ E_p = \frac{hc}{\lambda} $$

where E is photon energy, and h is Planck's constant.

Conversely, the number of photons at a given energy can be found by: $$ E_n = \frac{\lambda}{hc} $$

Therefore, if spectral radiance is: 
1 $W s^{-1} m^{-2} sr^{-1} nm^{-1}$ at 500nm (0.0005 m), this equates to:

In [9]:
print (str(round( (500 *10**-9) /(6.62607015 * 10**-34 * 299792458))) +' photons')

2517058283771355136 photons


Where $6.62607015 * 10^{-34} j/s$ is Planck's constant and 299792458 m/s is the speed of light.

Simplified, the number of photons per joule is: $$ N_{p \lambda} = \lambda * 5.03*10^{15}$$

In [11]:
print ('Photons per joule: '+str(500*(5.034117*10**15)))

Photons per joule: 2.5170585e+18


Since a Watt of power is 1 joule per second, this formulation can be used to 
convert Watt Radiance to photon radiance, by calculating this value for each
wavelength bin and factoring it by the power at that bin and summing these 
photon numbers. 

In [ ]:
df %>% mutate(quanta = wv * (5.034117 * 10^15) * specradW) -> df

Add columns for the normalized spectral radiance in Watts (norm.W) and
represented as photon number (norm.quanta).

In [ ]:
df %>% group_by(wv,material) %>% mutate(norm.W = specradW / max(specradW)) -> df
df %>% group_by(wv,material) %>% mutate(norm.quanta = quanta / max(quanta)) -> df
df %>% group_by(white,material) -> df
head(df)

### Summarize

Get the number of counts for each material and shade combination

In [ ]:
df %>% 
  group_by(material,white) %>%
  summarize(n = n()) -> df_summary

df_summary

# Sanity checks

Get standard deviation between the three replicates (rep) for each combination of factors
to find ones where the replicates differ from each other.

In [ ]:
df %>% 
  group_by(material,white,wv) %>%
  summarize(#variance = sd(specradW) ^2,
            #rel_var = variance / sum(variance) # this isn't working!!
            sd     = sd(specradW),
            rel_sd = sd(specradW)/mean(specradW)
            ) -> wv_summary

wv_summary %>% 
  ggplot(aes(x=wv, y=rel_sd,color=material)) + geom_point() + 
  xlim(400,700) + ylim(0,0.35) + theme_classic()

Find combinations with highest variance.

In [ ]:
wv_summary %>%
  group_by(material,white) %>%
  summarize(tote = sum(rel_sd)) %>% arrange(desc(tote)) -> tote_rel_sd

head(tote_rel_sd)

The quanta at each 3 nm wavelength bin.

In [ ]:
df %>% 
  filter(material=="Thick") %>% group_by(white) %>%
  ggplot(aes(x=wv,y=quanta,color=white)) + geom_point() + xlim(390,690) +
  theme_classic()

In [ ]:
df %>% 
  ggplot(aes(x=wv,y=norm.quanta,group = white,color=material,shape=material)) + 
       geom_point(alpha=0.5) + 
       xlim(400,650) + ylim(0,1) + theme_classic()

Not a huge difference between the two materials. In the wv region which is
probably most relevant (450-550 nm) the thicker material is consistently brighter
than the thinner one for the lightest shades. More importantly, the thicker one
is slightly whiteer or equivalent for the whiteest shades across the entire range
from 400-600nm. I will proceed with the thicker material on account of its
greater contrast in the relevant spectral range (and its toughness.) 

In [ ]:
df %>% 
  filter(material=="Thick") %>%
  ggplot(aes(x=wv,y=norm.quanta,group = white,color=rep)) + 
       geom_point(alpha=0.5,shape=16) + 
       xlim(400,675) + ylim(0,1) + theme_classic()

Enhance, zoom and enhance.

In [ ]:
df %>% 
  filter(material=="Thick") %>%
  ggplot(aes(x=wv,y=norm.quanta,group = white,color=rep)) + 
       geom_point(alpha=0.3,shape=16) + 
       xlim(400,600) + ylim(0,1) + theme_classic()

## First approximation to compare ratios of quanta between test sets

If the peak is around 500 nm. We factor the quanta by 3 because the bin width
is 3 nm.

In [ ]:
df %>% 
  filter(between(wv,400,600)) %>% group_by(material,white,rep) %>%
  summarize(tote_quanta = sum(quanta*binwidth)
            ) -> photon_n

photon_n %>% 
  ggplot(aes(x=white,y=tote_quanta,color=rep,shape=material)) + geom_point() +
  labs(plot.title ='Number of photons in the range 425 to 575 nm') + 
  geom_line() + theme_classic()

For the thick material, the dark 25 (white 0.75) rep III is awry, as is the dark
90 (white 0.1) rep III, as is rep II of dark 65 (white 0.35) for the thin material.
I will remove these. 

In [ ]:
df2 <- df
df2 <- df2[!(df2$material=="Thick" & df2$dark==25 & df2$rep=="III"),]
df2 <- df2[!(df2$material=="Thick" & df2$dark==85 & df2$rep=="III"),]
df2 <- df2[!(df2$material=="Thin"  & df2$dark==65 & df2$rep=="II"),]

Plot the revised data in a reasonable wv range.

In [ ]:
df2 %>%
  filter(between(wv,400,600)) %>% group_by(material,white,rep) %>%
  summarize(tote_quanta = sum(quanta*binwidth)        )  %>%
  group_by(material) %>% mutate(norm_quanta = tote_quanta/max(tote_quanta)
                                ) %>% 
  ggplot(aes(x=white,y=norm_quanta,color=rep,shape=material)) + geom_point() +
  labs(plot.title ='Number of photons in the range 400 to 575 nm') + 
  geom_line() + theme_classic()

The dataframe is summarized is follows:

In [ ]:
summary(df2)
write_delim(df2, "spec_data_curated.txt", delim="\t")